<a href="https://colab.research.google.com/github/gptix/DS-Unit-2-Linear-Models/blob/master/ASSIGNMENT_Jud_Taylor_REVISED_assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

# Assignment

## Set up for Import.

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

## Import useful code.

In [0]:
# Import userful code.
import sys
import warnings
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

import pandas as pd
import pandas_profiling
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.express as px

import category_encoders as ce

## Import raw data.

In [0]:
# Read New York City property sales data
df_raw = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

Make a non-raw copy of df.

In [0]:
df = df_raw

## Clean data.

In [0]:
# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

# df.columns

## Filter data.

In [0]:
# Use a subset of the data where BUILDING_CLASS_CATEGORY == 
# '01 ONE FAMILY DWELLINGS' and the sale price 
# was more than 100 thousand and less than 2 million.
# df_raw.shape
df = df[df['BUILDING_CLASS_CATEGORY']=='01 ONE FAMILY DWELLINGS']

df = df[(df['SALE_PRICE']<2e6) & (df['SALE_PRICE']>1e5)]

# df.columns
# df.shape

(3151, 21)

## Engineer some more.

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

## Take a look at data.

In [0]:
# Review data.
# df
# df.describe()
# df.describe().T
# df.head()
# df.tail()
# df.columns
# df.dtypes
# df['bedrooms'].describe()
df.shape
df.columns.isna()
# df.isna()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [0]:
df['SALE_DATE']
'01/01/2019' < '01/02/2019'

True

## Split for Train and Test


In [0]:
# Use data from January — March 2019 to train. 
df_train = df[(df['SALE_DATE'] >= '01/01/2019') & (df['SALE_DATE'] < '04/01/2019')]
df_train.shape

# Use data from April 2019 to test.
df_test = df[(df['SALE_DATE'] >= '04/01/2019') & (df['SALE_DATE'] < '05/01/2019')]
df_test.shape

(644, 21)

## Do one-hot encoding of categorical features.

In [0]:
# find categorical features.
df_train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
BUILDING_CLASS_CATEGORY,2507,1,01 ONE FAMILY DWELLINGS,2507
APARTMENT_NUMBER,1,1,RP.,1
TAX_CLASS_AT_PRESENT,2507,2,1,2476
BOROUGH,2507,5,4,1204
NEIGHBORHOOD,2507,11,OTHER,1911
BUILDING_CLASS_AT_TIME_OF_SALE,2507,11,A1,919
BUILDING_CLASS_AT_PRESENT,2507,13,A1,919
SALE_DATE,2507,68,01/31/2019,78
LAND_SQUARE_FEET,2507,887,"4,000",234
ADDRESS,2507,2497,118-20 202ND STREET,2


In [0]:
target = 'SALE_PRICE'

# Rows w/ only one value add no analytical value. 
only_one_value = ['BUILDING_CLASS_CATEGORY','APARTMENT_NUMBER']
high_cardinality = ['ADDRESS', 'LAND_SQUARE_FEET', 'SALE_DATE']

features = df_train.columns.drop([target] + high_cardinality + only_one_value)

df_train[features].T

,44,61,78,108,111,120,121,132,137,141,150,155,156,157,160,162,176,178,180,184,185,186,193,267,282,284,289,296,297,304,312,319,339,343,346,347,349,354,357,360,...,17992,17993,17997,18000,18001,18015,18016,18019,18026,18027,18034,18037,18038,18052,18075,18083,18085,18086,18095,18097,18101,18102,18105,18107,18110,18111,18112,18114,18116,18117,18120,18123,18124,18125,18128,18129,18130,18132,18134,18147
BOROUGH,3,4,2,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,5,5,5,5,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5,5,5,5,4
NEIGHBORHOOD,OTHER,QUEENS VILLAGE,OTHER,OTHER,OTHER,OTHER,OTHER,BAYSIDE,OTHER,FLUSHING-NORTH,OTHER,OTHER,OTHER,OTHER,OTHER,LAURELTON,SO. JAMAICA-BAISLEY PARK,OTHER,SPRINGFIELD GARDENS,OTHER,GREAT KILLS,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,FLUSHING-NORTH,FLUSHING-NORTH,...,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,FLUSHING-NORTH,FLUSHING-SOUTH,OTHER,OTHER,OTHER,OTHER,LAURELTON,OTHER,OTHER,SO. JAMAICA-BAISLEY PARK,SO. JAMAICA-BAISLEY PARK,SOUTH OZONE PARK,SPRINGFIELD GARDENS,ST. ALBANS,ST. ALBANS,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,GREAT KILLS,GREAT KILLS,GREAT KILLS,OTHER,OTHER,OTHER,OTHER,OTHER,QUEENS VILLAGE
TAX_CLASS_AT_PRESENT,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
BLOCK,5495,7918,4210,5212,7930,6806,6761,7378,2882,7407,10918,13962,14253,1103,3317,12908,12352,11936,12985,5708,5442,760,1448,8218,6243,4309,5142,4956,4984,8883,7306,8546,11305,1903,8276,8379,1516,15525,4848,5627,...,8647,15400,1967,8103,8106,8742,8826,5569,6933,3288,3655,10559,10866,13147,9535,13207,12012,12201,11656,12527,10409,11015,155,8916,1133,1156,1156,6220,5720,6022,5680,5442,5449,5458,54,4081,2373,1132,3395,11159
LOT,801,72,19,69,121,72,42,4,16,50,33,98,1518,57,24,45,463,56,48,35,145,89,1,68,6,36,115,4,68,34,68,47,19,21,600,130,44,13,35,21,...,34,85,50,56,18,159,25,16,50,18,31,13,60,39,45,57,183,64,50,70,28,58,28,54,35,1,82,17,90,103,32,269,63,102,61,44,201,42,37,11
EASE-MENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BUILDING_CLASS_AT_PRESENT,A9,A1,A1,A1,A5,A1,A1,A1,A5,A0,A5,A2,A1,A5,A3,A1,A1,A5,A1,A2,A9,A5,A2,A5,A5,A9,A1,A9,A5,A1,A5,A9,A5,A1,A2,A2,A1,S1,A2,A0,...,A5,A1,A5,A1,A1,A0,A1,A0,A5,A3,A9,A1,A1,A1,A1,A2,A1,A1,A1,A1,A2,A5,A5,A9,A5,A5,A5,A5,A5,A5,A9,A9,A9,A5,A1,A2,A5,A1,A2,A0
ZIP_CODE,11230,11427,10461,11226,11203,11229,11230,11364,11373,11365,11429,11414,11414,11369,11415,11413,11434,11435,11434,10312,10308,10314,10314,11236,11228,11207,11218,11203,11210,11229,11229,11234,11411,11373,11362,11362,11373,11691,11354,11365,...,11001,11693,11368,11363,11363,11004,11001,11365,11365,11375,11385,11428,11423,11413,11419,11422,11436,11434,11420,11434,11412,11412,11104,11421,11377,11377,11377,10312,10312,10312,10312,10308,10308,10308,10301,10306,10314,10302,10305,11429
RESIDENTIAL_UNITS,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
COMMERCIAL_UNITS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
df_train.describe(include='number').T

,count,mean,std,min,25%,50%,75%,max
BLOCK,2507.0,6758.303949,3975.909029,21.0,3837.5,6022.0,9888.5,16350.0
LOT,2507.0,75.778221,157.531138,1.0,21.0,42.0,70.0,2720.0
EASE-MENT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP_CODE,2507.0,10993.398484,494.291462,10301.0,10314.0,11234.0,11413.0,11697.0
RESIDENTIAL_UNITS,2507.0,0.987635,0.110532,0.0,1.0,1.0,1.0,1.0
COMMERCIAL_UNITS,2507.0,0.016354,0.129966,0.0,0.0,0.0,0.0,2.0
TOTAL_UNITS,2507.0,1.003989,0.171794,0.0,1.0,1.0,1.0,3.0
GROSS_SQUARE_FEET,2507.0,1473.744715,599.217635,0.0,1144.0,1368.0,1683.0,7875.0
YEAR_BUILT,2507.0,1944.766653,27.059337,1890.0,1925.0,1940.0,1960.0,2018.0
TAX_CLASS_AT_TIME_OF_SALE,2507.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0


In [89]:
df.columns[df_train.isna().any()].tolist()

['EASE-MENT', 'APARTMENT_NUMBER']

## Drop Columns (as features) that have NaN
These look unimportant.

In [91]:
features = df_train.columns.drop([target] + 
                                 high_cardinality + 
                                 only_one_value + 
                                 ['EASE-MENT', 'APARTMENT_NUMBER'])

features

df_train[features].T

,44,61,78,108,111,120,121,132,137,141,150,155,156,157,160,162,176,178,180,184,185,186,193,267,282,284,289,296,297,304,312,319,339,343,346,347,349,354,357,360,...,17992,17993,17997,18000,18001,18015,18016,18019,18026,18027,18034,18037,18038,18052,18075,18083,18085,18086,18095,18097,18101,18102,18105,18107,18110,18111,18112,18114,18116,18117,18120,18123,18124,18125,18128,18129,18130,18132,18134,18147
BOROUGH,3,4,2,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,5,5,5,5,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5,5,5,5,4
NEIGHBORHOOD,OTHER,QUEENS VILLAGE,OTHER,OTHER,OTHER,OTHER,OTHER,BAYSIDE,OTHER,FLUSHING-NORTH,OTHER,OTHER,OTHER,OTHER,OTHER,LAURELTON,SO. JAMAICA-BAISLEY PARK,OTHER,SPRINGFIELD GARDENS,OTHER,GREAT KILLS,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,FLUSHING-NORTH,FLUSHING-NORTH,...,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,FLUSHING-NORTH,FLUSHING-SOUTH,OTHER,OTHER,OTHER,OTHER,LAURELTON,OTHER,OTHER,SO. JAMAICA-BAISLEY PARK,SO. JAMAICA-BAISLEY PARK,SOUTH OZONE PARK,SPRINGFIELD GARDENS,ST. ALBANS,ST. ALBANS,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,OTHER,GREAT KILLS,GREAT KILLS,GREAT KILLS,OTHER,OTHER,OTHER,OTHER,OTHER,QUEENS VILLAGE
TAX_CLASS_AT_PRESENT,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
BLOCK,5495,7918,4210,5212,7930,6806,6761,7378,2882,7407,10918,13962,14253,1103,3317,12908,12352,11936,12985,5708,5442,760,1448,8218,6243,4309,5142,4956,4984,8883,7306,8546,11305,1903,8276,8379,1516,15525,4848,5627,...,8647,15400,1967,8103,8106,8742,8826,5569,6933,3288,3655,10559,10866,13147,9535,13207,12012,12201,11656,12527,10409,11015,155,8916,1133,1156,1156,6220,5720,6022,5680,5442,5449,5458,54,4081,2373,1132,3395,11159
LOT,801,72,19,69,121,72,42,4,16,50,33,98,1518,57,24,45,463,56,48,35,145,89,1,68,6,36,115,4,68,34,68,47,19,21,600,130,44,13,35,21,...,34,85,50,56,18,159,25,16,50,18,31,13,60,39,45,57,183,64,50,70,28,58,28,54,35,1,82,17,90,103,32,269,63,102,61,44,201,42,37,11
BUILDING_CLASS_AT_PRESENT,A9,A1,A1,A1,A5,A1,A1,A1,A5,A0,A5,A2,A1,A5,A3,A1,A1,A5,A1,A2,A9,A5,A2,A5,A5,A9,A1,A9,A5,A1,A5,A9,A5,A1,A2,A2,A1,S1,A2,A0,...,A5,A1,A5,A1,A1,A0,A1,A0,A5,A3,A9,A1,A1,A1,A1,A2,A1,A1,A1,A1,A2,A5,A5,A9,A5,A5,A5,A5,A5,A5,A9,A9,A9,A5,A1,A2,A5,A1,A2,A0
ZIP_CODE,11230,11427,10461,11226,11203,11229,11230,11364,11373,11365,11429,11414,11414,11369,11415,11413,11434,11435,11434,10312,10308,10314,10314,11236,11228,11207,11218,11203,11210,11229,11229,11234,11411,11373,11362,11362,11373,11691,11354,11365,...,11001,11693,11368,11363,11363,11004,11001,11365,11365,11375,11385,11428,11423,11413,11419,11422,11436,11434,11420,11434,11412,11412,11104,11421,11377,11377,11377,10312,10312,10312,10312,10308,10308,10308,10301,10306,10314,10302,10305,11429
RESIDENTIAL_UNITS,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
COMMERCIAL_UNITS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
TOTAL_UNITS,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,1,1,...,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


## Split train and test sets into X matrix and y vector

In [0]:
X_train = df_train[features]
y_train = df_train[target]
X_test = df_test[features]
y_test = df_test[target]

## Do One Hot Encoder

In [0]:
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)

In [97]:
X_train

,BOROUGH_3,BOROUGH_4,BOROUGH_2,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_QUEENS VILLAGE,NEIGHBORHOOD_BAYSIDE,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_LAURELTON,NEIGHBORHOOD_SO. JAMAICA-BAISLEY PARK,NEIGHBORHOOD_SPRINGFIELD GARDENS,NEIGHBORHOOD_GREAT KILLS,NEIGHBORHOOD_FLUSHING-SOUTH,NEIGHBORHOOD_SOUTH OZONE PARK,NEIGHBORHOOD_ST. ALBANS,TAX_CLASS_AT_PRESENT_1,TAX_CLASS_AT_PRESENT_1D,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT_A9,BUILDING_CLASS_AT_PRESENT_A1,BUILDING_CLASS_AT_PRESENT_A5,BUILDING_CLASS_AT_PRESENT_A0,BUILDING_CLASS_AT_PRESENT_A2,BUILDING_CLASS_AT_PRESENT_A3,BUILDING_CLASS_AT_PRESENT_S1,BUILDING_CLASS_AT_PRESENT_A4,BUILDING_CLASS_AT_PRESENT_A6,BUILDING_CLASS_AT_PRESENT_A8,BUILDING_CLASS_AT_PRESENT_B2,BUILDING_CLASS_AT_PRESENT_S0,BUILDING_CLASS_AT_PRESENT_B3,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
44,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,5495,801,1,0,0,0,0,0,0,0,0,0,0,0,0,11230.0,1.0,0.0,1.0,1325.0,1930.0,1,1,0,0,0,0,0,0,0,0,0,0
61,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,7918,72,0,1,0,0,0,0,0,0,0,0,0,0,0,11427.0,1.0,0.0,1.0,2001.0,1940.0,1,0,1,0,0,0,0,0,0,0,0,0
78,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,4210,19,0,1,0,0,0,0,0,0,0,0,0,0,0,10461.0,1.0,0.0,1.0,2043.0,1925.0,1,0,1,0,0,0,0,0,0,0,0,0
108,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,5212,69,0,1,0,0,0,0,0,0,0,0,0,0,0,11226.0,1.0,0.0,1.0,2680.0,1899.0,1,0,1,0,0,0,0,0,0,0,0,0
111,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,7930,121,0,0,1,0,0,0,0,0,0,0,0,0,0,11203.0,1.0,0.0,1.0,1872.0,1940.0,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,4081,44,0,0,0,0,1,0,0,0,0,0,0,0,0,10306.0,1.0,0.0,1.0,921.0,1950.0,1,0,0,0,0,1,0,0,0,0,0,0
18130,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,2373,201,0,0,1,0,0,0,0,0,0,0,0,0,0,10314.0,1.0,0.0,1.0,2128.0,1980.0,1,0,0,1,0,0,0,0,0,0,0,0
18132,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1132,42,0,1,0,0,0,0,0,0,0,0,0,0,0,10302.0,1.0,0.0,1.0,1807.0,2018.0,1,0,1,0,0,0,0,0,0,0,0,0
18134,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,3395,37,0,0,0,0,1,0,0,0,0,0,0,0,0,10305.0,1.0,0.0,1.0,621.0,1930.0,1,0,0,0,0,1,0,0,0,0,0,0
